<a href="https://colab.research.google.com/github/theaidran/ollama_youtube_summarize/blob/main/ollama_youtube_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Put a YouTube link and hit Run a cell.
Quick access to this colab notebook. [tinyurl.com/summarisation](https://tinyurl.com/summarisation)

In [1]:
# @title  { run: "auto", display-mode: "form" }
# All credits to https://github.com/martinopiaggi/summarize
# Good to see links:
# https://github.com/hsiehjackson/RULER
# https://ollama.com/library
# https://amgadhasan.substack.com/p/sota-asr-tooling-long-form-transcription


#1 Choose source url
Link = "https://www.youtube.com/watch?v=t6VoqbAh9HM" #@param {type:"string"}
Type_of_source = "Youtube video or playlist" #@param ['Youtube video or playlist', 'Google Drive video link','Dropbox video link', 'Local file']
Type = Type_of_source
URL = Link
Translate_to = "English" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}

#@markdown ---
#markdown Insert your API key depending on which endpoint you want to use
# API Configuration
api_key = "ollama"
api_endpoint = "Custom"
ollama_model = "llama3" # param ['llama3', 'phi3', 'vicuna:13b-v1.5-16k-q4_0']

endpoints = {
    "Groq": "https://api.groq.com/openai/v1",
    "OpenAI": "https://api.openai.com/v1",
    "Custom": "http://localhost:11434/v1"  # ollama endpoint
}
base_url = endpoints.get(api_endpoint)

models = {
    "Groq": "llama3-8b-8192",
    "OpenAI": "gpt-3.5-turbo",
    "Custom": ollama_model  # Placeholder for any custlom model
}
model = models.get(api_endpoint)

use_Youtube_captions = True #@param {type:"boolean"}
# @markdown  * otherwise transcribe audio


#2 launch ollama server with llama3
try: # to faster next start
  import pytube
except ImportError:
  !curl -fsSL https://ollama.com/install.sh | sh
  !tmux new -d ollama serve
  !ollama run {ollama_model} !


#3 install libs: youtube and whisperx
import subprocess
import re
import os
import IPython
from IPython.display import clear_output

if use_Youtube_captions:
  !pip install youtube-transcript-api
  from youtube_transcript_api import YouTubeTranscriptApi

if (not Type == "Youtube video or playlist") or (not use_Youtube_captions):
  try:
    import whisperx
  except ImportError:
    print("installing whisperx...may take a few minutes")
    !pip install git+https://github.com/m-bain/whisperx.git > /dev/null
    clear_output()

try:
  import deep_translator
except ImportError:
  !pip install -U deep-translator

try:
  import openai
except ImportError:
  !pip install openai
  #clear_output()

import openai
client = openai.OpenAI(api_key=api_key, base_url=base_url)

if Type == "Youtube video or playlist":
  try:
    import pytube
  except ImportError:
    #!pip install git+https://github.com/pytube/pytube
    !pip install git+https://github.com/metalshanked/pytube
    #clear_output()
    from pytube import YouTube

if Type == "Google Drive video link":
  from google.colab import drive
  drive.mount('/gdrive')

#4 Video fetching
skip_transcription=False
text = ""
textTimestamps = ""

def seconds_to_time_format(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"

def download_youtube_audio_only(url):
    yt = YouTube(url)
    audio_stream = yt.streams.get_audio_only()
    saved_path = audio_stream.download(output_path=".", skip_existing=True)
    return saved_path

def download_youtube_captions(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    try:
      transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
    except:
      for available_transcript in transcript_list:
        if available_transcript.is_translatable:
          transcript = available_transcript.translate('en').fetch()
          break

    text = ""
    for entry in transcript:
            start_time = seconds_to_time_format(entry['start'])
            text += f"{start_time} {entry['text'].strip()}\n"

    transcript_file_name = f"{video_id}_captions.md"

    with open(transcript_file_name, 'w', encoding='utf-8') as f:
      f.write(text)

    return text,transcript_file_name

if Type == "Youtube video or playlist":
    #clean youtube url from timestamp
    URL = re.sub('\&t=\d+s?', '', URL)
    if use_Youtube_captions:
      text, transcript_file_name = download_youtube_captions(URL)
      skip_transcription=True
    else:
      video_path_local =  download_youtube_audio_only(URL)

elif Type == "Google Drive video link":
  subprocess.run(['ffmpeg', '-y', '-i', '/gdrive/My Drive/' + URL, '-vn', '-acodec', 'pcm_s16le',
                  '-ar', '16000', '-ac', '1', 'gdrive_audio.wav'], check=True)
  video_path_local = "gdrive_audio.wav"

elif Type == "Dropbox video link":
    subprocess.run(['wget', URL, '-O', 'dropbox_video.mp4'], check=True)
    subprocess.run(['ffmpeg', '-y', '-i', 'dropbox_video.mp4', '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'dropbox_video_audio.wav'], check=True)
    video_path_local = "dropbox_video_audio.wav"

elif Type == "Local file":
    local_file_path = Source
    subprocess.run(['ffmpeg', '-y', '-i', local_file_path, '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'local_file_audio.wav'], check=True)
    video_path_local = "local_file_audio.wav"


#5 transcription using whisperx
if not skip_transcription:
  import whisperx
  language = "auto" # param {type:"string"}
  #initial_prompt = "" # @param {type:"string"}

  asr_options={"temperatures": 0, "beam_size": 1, "without_timestamps": True,}
  # Run on GPU with FP16
  modelWhisperX = whisperx.load_model("large-v2", device="cuda", compute_type="float16", asr_options=asr_options)

  transcription = modelWhisperX.transcribe(str(video_path_local), batch_size=16,
                                    language=None if language == "auto" else language,
                                    task="translate")

  transcript_file_name = os.path.splitext(video_path_local)[0]+'.md'

  with open(transcript_file_name, 'w') as f:
    for segment in transcription["segments"]:
        start_time = seconds_to_time_format(segment['start'])
        text += f"{start_time} {segment['text'].strip()} "

    f.write(text)


#6 Make a summary
prompt_type = "Summarization"  # param ['Summarization', 'Only grammar correction with highlights']
# markdown Set the number of parallel API calls (be mindful of usage rate limits)
parallel_api_calls = 1 # param


# Define your prompts using a dictionary for easier management
prompts = {
    'Summarization': """Summarize the video transcript excerpt including a concise title that reflects the content. Wrap the title with **markdown bold notation**. Write the summary as if you are continuing a conversation without needing to signal a beginning. Here is the transcript: """,
    'Only grammar correction with highlights': """Repeat the following text correcting any grammatical errors and formatting error. Highlight only the important quote (if there are any) with **markdown bold notation**. Focus solely on the essence of the content as if you are continuing a conversation without using any form of introduction like 'Here's the corrected text:'. Here is the text to fix: """
}

# Select the appropriate prompt
summary_prompt = prompts[prompt_type]



def extract_and_clean_timestamps(text_chunks):
    timestamp_pattern = re.compile(r'(\d{2}:\d{2}:\d{2})')
    cleaned_texts = []
    timestamp_ranges = []
    for chunk in text_chunks:
        timestamps = timestamp_pattern.findall(chunk)
        if timestamps:
            for timestamp in timestamps:
                # Remove each found timestamp from the chunk
                chunk = chunk.replace(timestamp, "")
            timestamp_ranges.append(timestamps[0])  # Assuming you want the first timestamp per chunk
        else:
            timestamp_ranges.append("")
        cleaned_texts.append(chunk.strip())  # Strip to remove any leading/trailing whitespace
    return cleaned_texts, timestamp_ranges

def format_timestamp_link(timestamp):
    if Type == "Youtube video or playlist":
      hours, minutes, seconds = map(int, timestamp.split(':'))
      total_seconds = hours * 3600 + minutes * 60 + seconds
      return f"{timestamp} - {URL}&t={total_seconds}"
    else:
      return f"{timestamp}"

import concurrent.futures
import time
from deep_translator import GoogleTranslator

def summarize(prompt):
    completion = client.chat.completions.create(
            model=model,
            messages=[
            {"role": "system", "content": summary_prompt},
            {"role": "user", "content": prompt}
            ],
            max_tokens=4096
    )
    return completion.choices[0].message.content

def process_and_summarize(text,chunk_size):
    overlap_size = 20
    texts = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - overlap_size)]
    cleaned_texts, timestamp_ranges = extract_and_clean_timestamps(texts)
    summaries = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=parallel_api_calls) as executor:
        future_to_chunk = {executor.submit(summarize, text_chunk): idx for idx, text_chunk in enumerate(cleaned_texts)}
        for future in concurrent.futures.as_completed(future_to_chunk):
            idx = future_to_chunk[future]
            try:
                summarized_chunk = future.result()
                summary_piece = format_timestamp_link(timestamp_ranges[idx]) + " " + summarized_chunk
                summary_piece += "\n"
                summaries.append((idx, summary_piece))
            except Exception as exc:
                print(f'Chunk {idx} generated an exception: {exc}')
                # Resubmit the task with the new model
                time.sleep(10)
                future_to_chunk[executor.submit(summarize, texts[idx])] = idx

    summaries.sort()  # Ensure summaries are in the correct order
    final_summary =""
    if Translate_to != "English":
      translated_summaries = []
      for _,summary in summaries:
        translated_summary = GoogleTranslator(source='auto', target=language_codes[Translate_to]).translate(summary)
        translated_summaries.append(translated_summary)
        final_summary += translated_summary + "\n\n"
      final_summary = final_summary.strip()
    else :
      final_summary = "\n\n".join([summary for _, summary in summaries])

    # Save the final summary
    final_name = transcript_file_name.replace(".md", "_FINAL_"+str(chunk_size)+".md") if Type != "Dropbox video link" else "final_dropbox_video.md"
    with open(final_name, 'w') as f:
        f.write(final_summary)
    return (final_summary)

from IPython.core.interactiveshell import InteractiveShell # Unblock output
InteractiveShell.ast_node_interactivity = "all"            # It starts to work after restart cell
#%config InteractiveShell.ast_node_interactivity="all"
clear_output()
print ("Generating summary...")
general_summary = process_and_summarize(text,15000) # max string size = max_tokens x 4
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary)
final_summary = process_and_summarize(text,3072)
clear_output()
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary + "\n\n" + "\n\n **Long Summary:** \n\n" + final_summary )





 **Short Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 **"Building a Website with AI: A Step-by-Step Guide"**

This video transcript excerpt provides a comprehensive guide on how to build a website using artificial intelligence (AI) tools. The speaker, who is creating a website that reviews camera drones, showcases several AI-powered tools such as Color Mind and Font Joy to help design the website's color scheme and font pairing.

The video begins with an overview of the website and its purpose: to compare and review camera drones to help consumers make informed decisions when purchasing. The speaker then uses Color Mind to generate a color scheme for the website, which is a crucial step in creating a visually appealing website.

Next, the speaker uses Font Joy to generate font pairing options for the website. This tool suggests font combinations that can enhance the overall design of the website.

The video also explores the process of writing copy for the website using an AI-powered tool called Claud. The speaker provides a prompt and instructions to write a heading and subheading in the style of Alex Horoi and David Ogoli.

Furthermore, the speaker demonstrates how to generate images for the website using mid-journey, a free online tool that can produce high-quality images based on user input. The video also covers how to use Canva to create featured images for blog posts.

Throughout the video, the speaker highlights the benefits of using AI-powered tools in building a website, including time-saving, increased accuracy, and enhanced design options. Overall, this guide provides valuable insights and practical tips for anyone looking to build a website with AI assistance.


00:11:15 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=675 **AI-Powered Featured Images: From Canva to a Professional Website**

In this conversation, we're continuing our discussion on building a website using AI tools. The speaker shows how to create a featured image using Canva and its AI features, such as Magic Media and background removers. They use these tools to generate images that match the style they want for their blog posts. Additionally, they highlight other AI tools used in this process, including font pairing, color schemes, and copywriting assistance from ChatGPT or Claude. The speaker also shares their experience with using Hostinger, Fontjoy, and Color Hunt to build a website, emphasizing the potential of AI in making the process faster and more efficient.




 **Long Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 **AI-Powered Website Building: A Game-Changer for Everyone**

You're continuing from where you left off, discussing AI-enabled tools that can help anyone get a website online quickly and easily without needing technical knowledge. You highlighted Reloom as one such tool, which uses AI to map out your entire website. You showed how to create a simple website using the free plan, and then generate content for each page using AI. With Reloom, you can build a comprehensive site map with individual sections for each page, and even get a rough idea of what each page could look like. This tool has the potential to democratize website creation, making it accessible to everyone regardless of their technical expertise.


00:02:18 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=138 **Building Your Website with AI**

So, you're not starting from scratch anymore. Now it's time for hosting, and I recommend Hostinger, who happens to be sponsoring today's video. As a result, they've hooked you up with a deal! Head over to hostinger.com to get your website for as little as $2.99/month, plus two months free. What's cool about Hostinger is that it'll actually build your website for you using AI on WordPress. Let's go ahead and click "Claim Deal" and select the Business website builder plan because most of the AI tools are under this plan. We can then add this to our cart, choosing whichever option works best for us.


00:04:34 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=274 **AI-Powered Design Inspiration**

We continue exploring AI-powered design tools, this time diving into Colormind and Font Joy. We start by generating a random color palette on Colormind, and then lock in the colors that resonate with us. From there, we explore how to apply these colors to our website's style section.

Next, we jump into Font Joy, an AI-powered font discovery tool. We generate various font styles until we find the perfect ones that complement our chosen color palette. In this case, we settle on the Roboto Condensed and Work Sans fonts.

We conclude by selecting these fonts in our website's style section, using them for headers and subheadings. This video demonstrates how AI can help designers discover design inspiration, streamline their workflow, and create visually appealing websites.


00:06:48 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=408 **Building a Website: Filling in Content with AI Help**

The conversation continues as the speaker discusses using AI tools, specifically ChatGPT or Claude, to generate content for their website. They explain that the goal is to create a prompt that gets the desired style of writing, and then use it to write a header and subheader in the style of Alex Horoi and David Ogoli. The speaker shares their experience with generating the heading and subheading using ChatGPT, which resulted in a clear and informative summary that can be used for their website's content.


00:09:01 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=541 **Creating Relevant Images for Website Content**

The video transcript excerpt continues with the creator discussing how to generate relevant images for website content. The speaker mentions using a DJI Mavic 2 camera drone to capture high-quality images and also explores alternative tools such as Mid Journey, Dolly 3, and Bing.com's image generation capabilities. The process involves tweaking settings, waiting for the images to generate, and replacing them on the website. The creator also recommends using Canva to add text, remove backgrounds, and enhance featured images for blog posts.


00:11:27 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=687 **Consistent Branding Through Custom Featured Images**

The conversation continues with discussing the importance of consistent branding through custom featured images. The speaker highlights the benefits of using Magic Media's built-in image generator in Canva to create a unified style for every blog post. They demonstrate how to use this feature, generating several images, and then select one that fits their needs. The speaker also shares how to remove the background from an existing image using Canva's AI-powered background remover, which is only available in the paid version of the software. Additionally, they discuss adding a white outline around themselves in the image to make it stand out and finally, they wrap up by suggesting using the Robotto Condensed font style for their text.


00:13:31 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=811 **Building a Website with AI: The Future of Web Development**

The summary continues as if we're already in the middle of a conversation:

...and now we've got a pretty quickly made featured image using Canva and various AI features. We download this image as a JPEG, jump back over to our website, change out our featured image to the one we just created, and resize it a little bit so it uses the full image. There's our featured image with the text from the blog post right here...


00:15:55 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=955 **Stay in the Loop with AI News and Tools**

The speaker continues to share a recent discovery, encouraging viewers to check out Future Tools, where they curate the coolest AI tools and update it almost daily. They also offer a free newsletter that shares only the most important news and tool recommendations. As an added incentive, the speaker will be giving away an Insta 360 X4 camera each month to subscribers of their YouTube channel and newsletter, starting from this point on.
